In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cmocean.cm as cmo
import xarray as xr
import ipywidgets as widgets
from ipywidgets import interactive_output, GridspecLayout, Layout

In [2]:
# Function to write the resulting bathymetry in a format that MITgcm understand (float32 is also possible as type)
def writefield(fname, data):
    import sys
    print('write to file: ' + fname)
    if sys.byteorder == 'little': data.byteswap(True)
    fid = open(fname, "wb")
    data.astype('float64').tofile(fid)
    fid.close()

## We need to construct the bathymetries for our idealized experiments

First, the depth axis. We load an axis created with `construct_vertical_grid.ipynb` following the method proposed by [Stewart et al., 2017](https://github.com/kialstewart/vertical_grid_for_ocean_models/blob/master/build_vertical_grid_kds.py). We need the depth axis to determine the maximum depth of our domain. (One could also set H_ to the desired value, if one does not have a depth axis yet.)

In [3]:
ds_z_axis = xr.open_dataset("vertical_grid.nc")
dz = ds_z_axis["drF"].values # thickness of cells
H_ = np.sum(dz)
H_int = int(H_)

Now, we define the different parameters and functions used to create the bathymetry

In [4]:
# All parameters that can be changed as widgets later on
first_col_w = "60%" # width of the first column of widgets
# the first group of parameters, the general settings: domain size, dx, dy, etc
x_num_cells = widgets.BoundedIntText(min=20, max=2000, value=250, step=5, layout=Layout(width=first_col_w), description="x_num_cells")
y_num_cells = widgets.BoundedIntText(min=20, max=2000, value=300, step=5, layout=Layout(width=first_col_w), description="y_num_cells")
dx = widgets.BoundedFloatText(min=0.1, max=100., value=10., step=0.1, layout=Layout(width=first_col_w), description="dx")
dy = widgets.BoundedFloatText(min=0.1, max=100., value=10., step=0.1, layout=Layout(width=first_col_w), description="dy")
wall = widgets.ToggleButton(value=True, description="add wall S", layout=Layout(width=first_col_w))
H = widgets.FloatSlider(min=H_, max=H_, value=H_)

res_col_w = "80%" # width of the second column of widgets
# second group of parameters to define the shelf
shelf = widgets.ToggleButton(value=True, description="add shelf", layout=Layout(width=res_col_w))
shelf_depth = widgets.IntSlider(min=0, max=H_int, value=300, step=10, description="depth", layout=Layout(width=res_col_w))
shelf_slope_position = widgets.IntSlider(min=10, max=2000, value=300, step=10, description="slope pos.", layout=Layout(width=res_col_w))
shelf_slope_width = widgets.IntSlider(min=10, max=1000, value=100, step=10, description="slope width", layout=Layout(width=res_col_w))

# third group of parameters to define the meridional ridge
meridional_ridge = widgets.ToggleButton(value=False, description="add meridional ridge", layout=Layout(width=res_col_w))
meridional_ridge_position= widgets.IntSlider(min=10, max=90, value=40, step=1, description="position", layout=Layout(width=res_col_w))
meridional_ridge_depth = widgets.IntSlider(min=0, max=H_int, value=0, step=10, description="depth", layout=Layout(width=res_col_w))
meridional_ridge_width = widgets.IntSlider(min=10, max=1000, value=200, step=10, description="width", layout=Layout(width=res_col_w))
meridional_ridge_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))

# fourth group of parameters to define the opening in the meridional ridge
meridional_opening = widgets.ToggleButton(value=False, description="add opening to ridge", layout=Layout(width=res_col_w))
meridional_opening_position = widgets.IntSlider(min=10, max=90, value=55, step=1, description="position", layout=Layout(width=res_col_w))
meridional_opening_depth = widgets.IntSlider(min=0, max=H_int, value=3000, step=10, description="depth", layout=Layout(width=res_col_w))
meridional_opening_width = widgets.IntSlider(min=10, max=2000, value=1000, step=10, description="width", layout=Layout(width=res_col_w))
meridional_opening_slope_width = widgets.IntSlider(min=10, max=500, value=50, step=10, description="slope width", layout=Layout(width=res_col_w))

# fifth group of parameters to define the zonal ridge
zonal_ridge = widgets.ToggleButton(value=False, description="add zonal ridge", layout=Layout(width=res_col_w))
zonal_ridge_depth = widgets.IntSlider(min=0, max=H_int, value=0, step=10, description="depth", layout=Layout(width=res_col_w))
zonal_ridge_length = widgets.IntSlider(min=10, max=1000, value=400, step=10, description="length", layout=Layout(width=res_col_w))
zonal_ridge_width = widgets.IntSlider(min=10, max=500, value=100, step=10, description="width", layout=Layout(width=res_col_w))
zonal_ridge_slope_width = widgets.IntSlider(min=10, max=500, value=100, step=10, description="slope width", layout=Layout(width=res_col_w))

Now we define a function that constructs a shelf and adds it to the Southern limit of the domain.  
The shelf is defined based on a hyperbolic tangent:  
$s(y) = H_s + 0.5 (H - H_s) (1 + \tanh(\frac{y - s_p}  {s_w}))$  
Where $H_s$ is the depth of the shelf, $H$ is the maximum depth of the domain, $y$ is the y distance in km, $s_p$ is the y position of the center of the slope and $s_w$ is the width of the slope.  
Both, $H_s$ and $H$ must be zero or negative.

In [5]:
def create_shelf(shelf, coords):
    """
    Adds a shelf at the southern boundary of the domain.
    
    Parameters
    ----------
    shelf : dict
        Dictionary containing the parameters needed to define the shelf
        -----
        shelf : bool
            If True, shelf is added. Default is False.
        shelf_depth : int
            Depth of the shelf in m
        shelf_slope_position : int
            Position of the slope (between shelf and deep ocean) in km from southern boundary
        shelf_slope_width : int
            Width of the slope
            
    coords : dict
        Dictionary containing the general parameters of the domain
        -----
        xnum : int
            Number of cells in x direction
        ynum : int
            Number of cells in y direction
        dx : float
            Grid spacing in x direction in km
        dy : float
            Grid spacing in y direction in km
        H : float
            Maximum depth of the vertical axis (lower cell face of the deepest cell)
            
    Returns
    -------
    bathy : array
        Array containing the bathymetry with the added shelf (depth coords["H"] elsewhere)
    """
    s = shelf
    # construct the shelf
    Hs = -s["shelf_depth"] # depth on the shelf in m
    shelf_slope_center = s["shelf_slope_position"] # slope center position in km
    shelf_slope_width = s["shelf_slope_width"] # slope half width in km
    shelf_out = (Hs + (0.5 * (coords["H"] - Hs)) 
                 * (1 + np.tanh((coords["y_in_km"] - shelf_slope_center) / shelf_slope_width)))
    return shelf_out[:, None] * np.ones((coords["ynum"], coords["xnum"]))

Now we define a function that constructs a meridional ridge and adds it to the existing bathymetry.
Analogously to the shelf, the ridge is defined based on two slopes of a hyperbolic tangent:  
$s(x) = H_r + 0.5 (H - H_r) (1 + \tanh(\frac{x - s_p}  {s_w}))$  
Where $H_r$ is the shallowest depth of the ridge, $H$ is the maximum depth of the domain, $x$ is the x distance in km, $s_p$ is the y position of the center of the slope and $s_w$ is the width of the slope.  
Both, $H_r$ and $H$ must be zero or negative.

In [6]:
def create_meridional_ridge(bathy, meridional_ridge, coords):
    """
    Adds a meridional ridge to the domain.
    
    Parameters
    ----------
    bathy : array
        Array containing the original bathymetry
        
    meridional_ridge : dict
        Dictionary containing the parameters needed to define the meridional ridge
        -----
        meridional_ridge : bool
            If True, meridional ridge is added. Default is False.
        meridional_ridge_position : int
            Position of the meridional ridge in x direction in percent
            0 is at the western boundary, 100 at the eastern boundary
        meridional_ridge_depth : int
            Depth of the shallowest point of the meridional ridge in m
        meridional_ridge_width : int
            Width (west to east) of the meridional ridge
        meridional_ridge_slope_width : int
            Width of the (west to east) slope
            
    coords : dict
        Dictionary containing the general parameters of the domain
        -----
        xnum : int
            Number of cells in x direction
        ynum : int
            Number of cells in y direction
        dx : float
            Grid spacing in x direction in km
        dy : float
            Grid spacing in y direction in km
        H : float
            Maximum depth of the vertical axis (lower cell face of the deepest cell)
    
    Returns
    -------
    bathy : array
        Array containing the original bathymetry with the meridional ridge added
    """
    m = meridional_ridge
    # define the positions of the meridional ridge
    mr_pos_per = m["meridional_ridge_position"]
    mr_pos = int(coords["xnum"] *  mr_pos_per / 100)
    H_ridge = -m["meridional_ridge_depth"] # shallowest depth of the ridge in m
    Hr = H_ridge + 100 # adjust minimum depth to allow for small deviations and still reach minimum depth
    mr_center =  mr_pos * coords["dx"] # x position of the ridge in km
    mr_slope_width = m["meridional_ridge_slope_width"] # width of the slope of the ridge in km
    mr_width = m["meridional_ridge_width"] # width of the ridge in km
    mr_width_cells = int(mr_width / coords["dx"]) # width of the ridge in cells
    mr_start = mr_pos - int(mr_width_cells / 2) # x position where to start the western slope of the ridge
    # ridge_slope_west is the western part of the slope
    mr_slope_west = Hr + ((0.5 * (coords["H"] - Hr)) *
                          (1 + np.tanh((coords["x_in_km"][mr_start::] - mr_center - mr_slope_width) 
                           / mr_slope_width)))
    # ridge_slope_east is the eastern part of the slope 
    mr_slope_east = Hr + ((0.5 * (coords["H"] - Hr)) *
                          (1 + np.tanh((coords["x_in_km"][mr_start::] - mr_center - mr_slope_width)
                            / mr_slope_width)))
    mr_tmp = np.hstack((mr_slope_west[::-1], mr_slope_east))[0:coords["xnum"]] # put together both slopes
    # if the array with the ridge is two short, add values H to eastern end
    mr = np.ones(coords["xnum"]) * coords["H"]
    mr[0:len(mr_tmp)] = mr_tmp
    mr = mr[::-1] # flip around the array
    mr_fr = mr / coords["H"] # write the ridge as a fraction of maximum depth H
    return bathy * mr_fr[None, :] # add ridge to existing bathumetry

Now we create a function to add an opening to the meridional ridge. The slopes of this opening are again defined based on a hyperbolic tangent like above.

In [7]:
# function to add opening to meridional ridge
def create_meridional_opening(bathy, meridional_ridge, meridional_opening, coords):
    """
    Adds an opening to the meridional ridge to the domain.
    Requires that a meridional ridge has been added before.
    
    Parameters
    ----------
    bathy : array
        Array containing the original bathymetry
        
    meridional_ridge : dict
        Dictionary containing the parameters needed to define the meridional ridge
        Needed only to extract the depth of the meridional ridge
        -----
        meridional_ridge_depth : int
            Depth of the shallowest point of the meridional ridge in m

    
    meridional_opening : dict
        Dictionary containing the parameters needed to define an opening 
        -----
        meridional_opening : bool
            If True, opening is added. Default is False.
        meridional_opening_position : int
            Position of the opening in y direction in percent
            0 is at the southern boundary, 100 at the northern boundary
        meridional_opening_depth : int
            Depth of the shallowest point of the opening in m
        meridional_opening_width : int
            Width (south to north) of the opening
        meridional_opening_slope_width : int
            Width of the (south to north) slopes at the southern and northern boundaries of the opening
            
    coords : dict
        Dictionary containing the general parameters of the domain
        -----
        xnum : int
            Number of cells in x direction
        ynum : int
            Number of cells in y direction
        dx : float
            Grid spacing in x direction in km
        dy : float
            Grid spacing in y direction in km
        H : float
            Maximum depth of the vertical axis (lower cell face of the deepest cell)
    
    Returns
    -------
    bathy : array
        Array containing the original bathymetry with an opening added to the meridional ridge
    """
    m = meridional_opening
    H_ridge = -meridional_ridge["meridional_ridge_depth"]
    mr_open_pos_per = m["meridional_opening_position"]
    mr_open = int(coords["ynum"] *  mr_open_pos_per / 100)
    Ho = -m["meridional_opening_depth"] # depth of the opening in m
    mr_open_center = mr_open * coords["dy"] # center positiop of the opening in km
    mr_open_slope_width = m["meridional_opening_slope_width"] # slope width in km of the opening (probably good to set it to mr_slope_width)
    mr_open_width = m["meridional_opening_width"] # latitudinal width of the opening in km
    mr_open_width_cells = int(mr_open_width / coords["dy"]) # width of opening in number of cells
    # translate the information to a km range where the opening should be located
    mr_range = coords["y_in_km"][(mr_open - int(mr_open_width_cells / 2)):(mr_open + int(mr_open_width_cells / 2))]
    # construct the slope of the opening
    mr_open1 = H_ridge + ((0.5 * (Ho - H_ridge)) *
               (1 + np.tanh((mr_range - mr_open_center - (mr_open_slope_width))
                            / mr_open_slope_width)))
    mr_open2 = np.zeros(coords["ynum"])
    # create the opening
    index1 = mr_open_width_cells - (len(range((mr_open - mr_open_width_cells), mr_open)) - len(mr_open1))
    mr_open2[(mr_open - index1):mr_open] = mr_open1 # southern slope of the opening
    mr_open2[mr_open:(mr_open + index1)] = mr_open1[::-1] # northern slope of the opening
    mr_open2 -= H_ridge
    bathy2 = bathy + mr_open2[:, None] # add opening to bathymetry
    bathy[bathy>bathy2] = bathy2[bathy>bathy2]
    bathy[bathy < coords["H"]] = coords["H"] # restore everything that got distorted to the maximum depth H
    return bathy

And finally, a function to add a zonal ridge that starts at the southern end of the opening of the meridional ridge. The northern and southern slopes of this ridge are again described by a the same hyperbolic tangent. The slope at the eastern end of the ridge is defined by a cosine:  
$s(x, y) = (z_r(y) - H)\cos(x) $  
Where $z_r(y)$ is the original depth of the zonal ridge at y position $y$, $H$ is the maximum depth of the domain and $x=[0,...\pi]$

In [8]:
def create_zonal_ridge(bathy, meridional_ridge, meridional_opening, zonal_ridge, coords):
    """
    Adds a zonal ridge to the existing meridional ridge.
    Requires that an opening of the meridional ridge has been added before.
    
    Parameters
    ----------
    bathy : array
        Array containing the original bathymetry
        
    meridional_ridge : dict
        Dictionary containing the parameters needed to define the meridional ridge
        Needed only to extract the position and width of the slope of the meridional ridge
        -----
        meridional_ridge_position : int
            Position of the meridional ridge in x direction in percent
        meridional_ridge_slope_width : int
            Width of the (west to east) slope

    meridional_opening : dict
        Dictionary containing the parameters needed to define an opening 
        Needed only to extract the position and width of the opening
        -----e.
        meridional_opening_position : int
            Position of the opening in y direction in percent
            0 is at the southern boundary, 100 at the northern boundary
        meridional_opening_width : int
            Width (south to north) of the opening
            
    zonal_ridge : dict
        Dictionary containing the parameters needed to define the zonal ridge
        zonal_ridge : bool
            If True, zonal ridge is added. Default is False.
        zonal_ridge_depth : int
            Depth of the shallowest point of the zonal ridge in m
        zonal_ridge_length : int
            Length of the zonal ridge in x direction in km
        zonal_ridge_slope_width : int
            Width of the slope (west to east) of the eastern end of the zonal ridge
        zonal_ridge_width : int
            Width (south to north) of the zonal ridge
            
    coords : dict
        Dictionary containing the general parameters of the domain
        -----
        xnum : int
            Number of cells in x direction
        ynum : int
            Number of cells in y direction
        dx : float
            Grid spacing in x direction in km
        dy : float
            Grid spacing in y direction in km
        H : float
            Maximum depth of the vertical axis (lower cell face of the deepest cell)
    
    Returns
    -------
    bathy : array
        Array containing the original bathymetry with an opening added to the meridional ridge
    """
    m = meridional_ridge
    mo = meridional_opening
    z = zonal_ridge
    # add zonal ridge
    Hz = -z["zonal_ridge_depth"] # depth of zonal ridge in m
    zr_len = z["zonal_ridge_length"] # length of the zonal ridge in km
    zr_zonal_slope = z["zonal_ridge_slope_width"] # width of the slope at the eastern end of ridge
    zr_slope_width = m["meridional_ridge_slope_width"] # width in the north-south direction
    zr_width_cells = int(z["zonal_ridge_width"] / coords["dx"]) # width in cells
    mr_open_pos_per = mo["meridional_opening_position"] # position of the opening in the meridional ridge
    mr_open = int(coords["ynum"] *  mr_open_pos_per / 100)
    zr_start = mr_open - int((mo["meridional_opening_width"] / coords["dy"]) / 2) # find the southern end of the opening in the meridional ridge
    zr_plus = zr_start + int(zr_width_cells) # add range around zr_start depending on how wide the zonal ridge should be
    zr_minus = zr_start - int(zr_width_cells)
    mr_pos_per = m["meridional_ridge_position"] # find out where to start zonal ridge in x direction
    mr_pos = int(coords["xnum"] *  mr_pos_per / 100)
    zr_x = int((mr_pos + zr_len / coords["dx"])) # start point (in x) of zonal ridge
    zr_x2 = int(zr_x + zr_zonal_slope / coords["dx"]) # end point (in x) of zonal ridge
    # construct the north and south slopes of the ridge
    zr1 = Hz + ((0.5 * (coords["H"] - Hz)) *
                (1 + np.tanh((coords["y_in_km"][zr_start:zr_plus] - zr_start * coords["dx"] - zr_slope_width) 
                             / zr_slope_width)))
    zr2 = np.hstack((zr1[::-1], zr1))
    # add the ridge to a dummy bathymetry and then add that to the original bathymetry in  applicable locations
    bathy2 = np.ones((coords["ynum"], coords["xnum"])) * coords["H"]
    bathy2[zr_minus:zr_plus, mr_pos:zr_x] += (zr2 - coords["H"])[:, None]
    # add a slope at the eastern end of the zonal ridge
    bathy2[zr_minus:zr_plus, zr_x:zr_x2] += ((zr2 - coords["H"])[:, None] 
                                             * np.cos(np.linspace(0, np.pi, int(zr_zonal_slope / coords["dx"]))))
    bathy[bathy<bathy2] = bathy2[bathy<bathy2]
    return bathy

Next, we need a function that creates a basic basin (flat bottom) and then calls the functions that add the different features if we want to add them.

In [9]:
def create_bat(x_num_cells, y_num_cells, dx, dy, max_depth, wall, shelf, meridional_ridge, meridional_opening, zonal_ridge):
    """
    Creates a dictionary with all the general parameters of the domain, sets up a simple, flat-bottom basin of depth max_depth,
    then calls the functions to create the different features of the bathymetry if required and returns the bathymetry.
    
    Parameters
    ----------
    x_num_cells : int
        Number of cells in x direction
    y_num_cells : int
        Number of cells in y direction
    dx : float
        Grid spacing in x direction in km
    dy : float
        Grid spacing in y direction in km
    max_depth : float
        Maximum depth of the vertical axis (lower cell face of the deepest cell)
    wall : bool
        If True, adds a solid wall (land) to the southern boundary of the domain
    shelf : dict
        Dictionary containing the parameters needed to define the shelf
        -----
        shelf : bool
            If True, shelf is added. Default is False.
        shelf_depth : int
            Depth of the shelf in m
        shelf_slope_position : int
            Position of the slope (between shelf and deep ocean) in km from southern boundary
        shelf_slope_width : int
            Width of the slope
    meridional_ridge : dict
        Dictionary containing the parameters needed to define the meridional ridge
        -----
        meridional_ridge : bool
            If True, meridional ridge is added. Default is False.
        meridional_ridge_position : int
            Position of the meridional ridge in x direction in percent
            0 is at the western boundary, 100 at the eastern boundary
        meridional_ridge_depth : int
            Depth of the shallowest point of the meridional ridge in m
        meridional_ridge_width : int
            Width (west to east) of the meridional ridge
        meridional_ridge_slope_width : int
            Width of the (west to east) slope
    meridional_opening : dict
        Dictionary containing the parameters needed to define an opening 
        -----
        meridional_opening : bool
            If True, opening is added. Default is False.
        meridional_opening_position : int
            Position of the opening in y direction in percent
            0 is at the southern boundary, 100 at the northern boundary
        meridional_opening_depth : int
            Depth of the shallowest point of the opening in m
        meridional_opening_width : int
            Width (south to north) of the opening
        meridional_opening_slope_width : int
            Width of the (south to north) slopes at the southern and northern boundaries of the opening
    zonal_ridge : dict
        Dictionary containing the parameters needed to define the zonal ridge
        zonal_ridge : bool
            If True, zonal ridge is added. Default is False.
        zonal_ridge_depth : int
            Depth of the shallowest point of the zonal ridge in m
        zonal_ridge_length : int
            Length of the zonal ridge in x direction in km
        zonal_ridge_slope_width : int
            Width of the slope (west to east) of the eastern end of the zonal ridge
        zonal_ridge_width : int
            Width (south to north) of the zonal ridge
            
    Returns
    -------
    bathy : array
        Array containing the bathymetry with the added features
    """
    coords = {}
    # number of cells in x and y directions
    coords["xnum"] = x_num_cells
    coords["ynum"] = y_num_cells
    # horizontal grid spacing
    coords["dx"] = dx
    coords["dy"] = dy
    # maximum depth of bathymetry
    coords["H"] = -max_depth
    # simple bathymetry, depth H everywhere
    #print("Creating basin with depth: ", max_depth, " m and dimensions x: ", x_num_cells, ", y: ", y_num_cells)
    bathy = np.ones((coords["ynum"], coords["xnum"])) * coords["H"]
    # the coordinates of the cell centers in km
    coords["x_in_km"] = np.linspace(int(coords["dx"]/2), coords["xnum"] * coords["dx"] - int(coords["dx"]/2), coords["xnum"])
    coords["y_in_km"] = np.linspace(int(coords["dy"]/2), coords["ynum"] * coords["dy"] - int(coords["dy"]/2), coords["ynum"])
    # add different features to bathymetry if required
    if shelf["shelf"]:
        #print("Adding a shelf with depth:", shelf["shelf_depth"], "m")
        bathy = create_shelf(shelf, coords)
    if meridional_ridge["meridional_ridge"]:
        #print("Adding a meridional ridge with depth:", meridional_ridge["meridional_ridge_depth"], "m at x=", int(coords["xnum"] / meridional_ridge["meridional_ridge_position"]))
        bathy = create_meridional_ridge(bathy, meridional_ridge, coords)
    if meridional_opening["meridional_opening"]:
        if meridional_ridge["meridional_ridge"]:
            #print("Adding an opening to the meridional ridge with depth:", meridional_opening["meridional_opening_depth"], "m and width",
            #      meridional_opening["meridional_opening_width"], "km at y=", int(coords["ynum"] / meridional_opening["meridional_opening_position"]), "km")
            bathy = create_meridional_opening(bathy, meridional_ridge, meridional_opening, coords)
        else:
            print("No opening added to meridional ridge, because no meridional ridge was defined")
    if zonal_ridge["zonal_ridge"]:
        if meridional_ridge["meridional_ridge"] & meridional_opening["meridional_opening"]:
            #print("Adding a zonal ridge with depth:", zonal_ridge["zonal_ridge_depth"], "at the southern slope of the opening in the meridional ridge")
            bathy = create_zonal_ridge(bathy, meridional_ridge, meridional_opening, zonal_ridge, coords)
        else:
            print("No zonal ridge added to meridional ridge, because no meridional ridge was defined")
    if wall:
        bathy[0, :] = 0 # construct a wall at the southern boundary
    bathy[bathy > -dz[0:3].sum()] = 0 # make sure there are at least three cells in the vertical everywhere where there is water
    return bathy
    

Now we create the dictionaries containing all the parameters needed for the functions that create the bathymetry. The inputs to `plot_bathy` are the widgets defined above, the values chosen by the user with the widgets will be forwarded to the dictionaries (`shelf_dict`, `meridional_ridge_dict`, ...) and a bathymetry will be created based on these values and then plotted.

In [10]:
def plot_bathy(x_num_cells=x_num_cells, y_num_cells=y_num_cells, dx=dx, dy=dy, max_depth=H, wall=wall,
               shelf=shelf, shelf_depth=shelf_depth, shelf_slope_position=shelf_slope_position, shelf_slope_width=shelf_slope_width,
               meridional_ridge=meridional_ridge, meridional_ridge_position=meridional_ridge_position, meridional_ridge_depth=meridional_ridge_depth,
               meridional_ridge_slope_width=meridional_ridge_slope_width, meridional_ridge_width=meridional_ridge_width,
               meridional_opening=meridional_opening, meridional_opening_position=meridional_opening_position, meridional_opening_depth=meridional_opening_depth,
               meridional_opening_slope_width=meridional_opening_slope_width, meridional_opening_width=meridional_opening_width,
               zonal_ridge=zonal_ridge, zonal_ridge_depth=zonal_ridge_depth, zonal_ridge_length=zonal_ridge_length, 
               zonal_ridge_width=zonal_ridge_width, zonal_ridge_slope_width=zonal_ridge_slope_width):
    shelf_dict = {}
    shelf_dict["shelf"] = shelf
    shelf_dict["shelf_depth"] = shelf_depth
    shelf_dict["shelf_slope_position"] = shelf_slope_position
    shelf_dict["shelf_slope_width"] = shelf_slope_width
    #
    meridional_ridge_dict = {}
    meridional_ridge_dict["meridional_ridge"] = meridional_ridge
    meridional_ridge_dict["meridional_ridge_position"] = meridional_ridge_position
    meridional_ridge_dict["meridional_ridge_depth"] = meridional_ridge_depth
    meridional_ridge_dict["meridional_ridge_slope_width"] = meridional_ridge_slope_width
    meridional_ridge_dict["meridional_ridge_width"] = meridional_ridge_width
    #
    meridional_opening_dict = {}
    meridional_opening_dict["meridional_opening"] = meridional_opening
    meridional_opening_dict["meridional_opening_position"] = meridional_opening_position
    meridional_opening_dict["meridional_opening_depth"] = meridional_opening_depth
    meridional_opening_dict["meridional_opening_slope_width"] = meridional_opening_slope_width
    meridional_opening_dict["meridional_opening_width"] = meridional_opening_width
    #
    zonal_ridge_dict = {}
    zonal_ridge_dict["zonal_ridge"] = zonal_ridge
    zonal_ridge_dict["zonal_ridge_depth"] = zonal_ridge_depth
    zonal_ridge_dict["zonal_ridge_length"] = zonal_ridge_length
    zonal_ridge_dict["zonal_ridge_slope_width"] = zonal_ridge_slope_width
    zonal_ridge_dict["zonal_ridge_width"] = zonal_ridge_width
    #
    bathy = create_bat(x_num_cells, y_num_cells, dx, dy, max_depth, wall, shelf_dict, meridional_ridge_dict, meridional_opening_dict, zonal_ridge_dict)
    plt.figure(figsize=(10,8))
    plt.pcolormesh(np.arange(0, x_num_cells), np.arange(0, y_num_cells), bathy, cmap=cmo.deep_r, shading="auto")
    cb = plt.colorbar(pad=0.01)
    cb.set_label("m", fontsize="18")
    cb.ax.tick_params(labelsize="18")
    plt.contour(np.arange(0, x_num_cells), np.arange(0, y_num_cells), bathy, levels=np.arange(-4001, 1, 1000), colors="k")
    plt.title("Bathymetry", fontsize="30", pad=10)
    #plt.xticks(np.arange(0, x_num_cells, 5 * (100/dx)), np.arange(0, 2001, 500), fontsize="18");
    plt.xlabel("km", fontsize="18")
    #plt.yticks(np.arange(5 * (100/dy), y_num_cells, 5 * (100/dy)), np.arange(500, 2501, 500), fontsize="18");
    plt.ylabel("km", fontsize="18")
    

Now we define the layout of the widgets.

In [11]:
interactive_plot = interactive_output(plot_bathy, {"x_num_cells": x_num_cells, "y_num_cells": y_num_cells, "dx": dx, "dy": dy, "max_depth": H, "wall": wall,
               "shelf": shelf, "shelf_depth": shelf_depth, "shelf_slope_position": shelf_slope_position, "shelf_slope_width": shelf_slope_width,
               "meridional_ridge": meridional_ridge, "meridional_ridge_position": meridional_ridge_position, "meridional_ridge_depth": meridional_ridge_depth,
               "meridional_ridge_slope_width": meridional_ridge_slope_width, "meridional_ridge_width": meridional_ridge_width,
               "meridional_opening": meridional_opening, "meridional_opening_position": meridional_opening_position, "meridional_opening_depth": meridional_opening_depth,
               "meridional_opening_slope_width": meridional_opening_slope_width, "meridional_opening_width": meridional_opening_width,
               "zonal_ridge": zonal_ridge, "zonal_ridge_depth": zonal_ridge_depth, "zonal_ridge_length": zonal_ridge_length, 
               "zonal_ridge_width": zonal_ridge_width, "zonal_ridge_slope_width": zonal_ridge_slope_width})

grid = GridspecLayout(20, 5)
grid[0, 0] = x_num_cells; grid[1, 0] = y_num_cells; grid[2, 0] = dx; grid[3, 0] = dy; grid[4, 0] = wall

grid[0, 1:3] = shelf; grid[1, 1:3] = shelf_depth; grid[2, 1:3] = shelf_slope_position; grid[3, 1:3] = shelf_slope_width

grid[0, 3:5] = meridional_ridge; grid[1, 3:5] = meridional_ridge_position; grid[2, 3:5] = meridional_ridge_depth
grid[3, 3:5] = meridional_ridge_width; grid[4, 3:5] = meridional_ridge_slope_width

grid[6, 3:5] = meridional_opening; grid[7, 3:5] = meridional_opening_position; grid[8, 3:5] = meridional_opening_depth
grid[9, 3:5] = meridional_opening_width; grid[10, 3:5] = meridional_opening_slope_width

grid[12, 3:5] = zonal_ridge; grid[13, 3:5] = zonal_ridge_depth; grid[14, 3:5] = zonal_ridge_length; 
grid[15, 3:5] = zonal_ridge_width; grid[16, 3:5] = zonal_ridge_slope_width

grid[5:20, 0:3] = interactive_plot

display(grid)

GridspecLayout(children=(BoundedIntText(value=250, description='x_num_cells', layout=Layout(grid_area='widget0…

And again we create dictionaries containing all the final values that we chose from the interactive part to create the "final" bathymetry, i.e. the one that will be written to file. 

In [12]:
shelf_dict = {}
shelf_dict["shelf"] = shelf.value
shelf_dict["shelf_depth"] = shelf_depth.value
shelf_dict["shelf_slope_position"] = shelf_slope_position.value
shelf_dict["shelf_slope_width"] = shelf_slope_width.value
#
meridional_ridge_dict = {}
meridional_ridge_dict["meridional_ridge"] = meridional_ridge.value
meridional_ridge_dict["meridional_ridge_position"] = meridional_ridge_position.value
meridional_ridge_dict["meridional_ridge_depth"] = meridional_ridge_depth.value
meridional_ridge_dict["meridional_ridge_slope_width"] = meridional_ridge_slope_width.value
meridional_ridge_dict["meridional_ridge_width"] = meridional_ridge_width.value
#
meridional_opening_dict = {}
meridional_opening_dict["meridional_opening"] = meridional_opening.value
meridional_opening_dict["meridional_opening_position"] = meridional_opening_position.value
meridional_opening_dict["meridional_opening_depth"] = meridional_opening_depth.value
meridional_opening_dict["meridional_opening_slope_width"] = meridional_opening_slope_width.value
meridional_opening_dict["meridional_opening_width"] = meridional_opening_width.value
#
zonal_ridge_dict = {}
zonal_ridge_dict["zonal_ridge"] = zonal_ridge.value
zonal_ridge_dict["zonal_ridge_depth"] = zonal_ridge_depth.value
zonal_ridge_dict["zonal_ridge_length"] = zonal_ridge_length.value
zonal_ridge_dict["zonal_ridge_slope_width"] = zonal_ridge_slope_width.value
zonal_ridge_dict["zonal_ridge_width"] = zonal_ridge_width.value
#
bathy = create_bat(x_num_cells.value, y_num_cells.value, dx.value, dy.value, H.value, wall, 
                   shelf_dict, meridional_ridge_dict, meridional_opening_dict, zonal_ridge_dict)

Here we add a sponge layer of 200km width to the northern boundary, as is commonly done in reentrant channel configurations. In the first 100km adjacent to the boundary, the restoring factor will be 1, in the next 100km only 0.25.

In [13]:
# thickness of sponge layer to build the restoring mask
sponge = int(100 / dy.value) 

mask = np.zeros((1, len(dz), y_num_cells.value, x_num_cells.value))
mask[:, 1:len(dz) + 1, -sponge::, :] = 1.0
mask[:, 1:len(dz) + 1, -(sponge * 2):-sponge, :] = 0.25

In [14]:
setup = "SOME_NAME"
confi = "CONFIG1"

In [15]:
writefield(setup + '-' + confi + '-bathymetry.bin', bathy)
writefield(setup + '-' + confi + '-restoring_mask_with_sponge.bin', mask)

write to file: SOME_NAME-CONFIG1-bathymetry.bin
write to file: SOME_NAME-CONFIG1-restoring_mask_with_sponge.bin
